In [1]:
import json
import heapq
from PIL import Image, ImageDraw
import os
import random
import numpy as np

In [27]:
### Config
data_dir = 'data/'
json_file = 'pretrained.json'
json_tracks = 'data/val_tracks_nlpaug.json'
output = 'report_attachment'

In [18]:
### Metric Functions

def recall(restuls, at=5):
    c = 0
    RR = 0
    for k in results:
        resultK = results[k]
        for i in range(at):
            if resultK[i] == k:
                RR += 1
                break
        c += 1
    return RR/c

def MRR(results):
    c = 0
    RR = 0
    for k in results:
        resultK = results[k]
        rank = 1
        for i in range(len(resultK)):
            if resultK[i] == k:
                break
            rank += 1
        RR += 1/rank
        c += 1
    return RR/c

def best(results, n=20):
    q = []
    heapq.heapify(q)

    for k in results:
        resultK = results[k]
        rank = 1
        for i in range(len(resultK)):
            if resultK[i] == k:
                break
            rank += 1
        heapq.heappush(q, (-rank, k))
        if len(q) > n:
            heapq.heappop(q)
    return q

def worst(results, n=20):
    q = []
    heapq.heapify(q)

    for k in results:
        resultK = results[k]
        rank = 1
        for i in range(len(resultK)):
            if resultK[i] == k:
                break
            rank += 1
        heapq.heappush(q, (rank, k))
        if len(q) > n:
            heapq.heappop(q)
    return q

def draw_bounding_box(
    img,
    box,
    fill=None,
    width=1,
):  
    ImageDraw.Draw(img).rectangle(box, fill=fill, outline='red', width=width)
    return img

In [19]:
def save_analysis(files, tracks, name='worst'):
    nl = dict()

    for index, (rank, k) in enumerate(files):
        img_bk = Image.open(f'data/motion_map/{k}.jpg')
        img_bk.save(os.path.join(output, f'{name}_{rank}_{index}_motion.png'))
        frames = tracks[k]['frames']
        ind =len(frames)//2 - 1
        img = Image.open(os.path.join(data_dir, frames[ind]))
        box = tracks[k]['boxes'][ind]
        box_align = [box[0], box[1], box[0] + box[2], box[1] + box[3]]
        draw_bounding_box(img, box_align,  width=6)
        img.save(os.path.join(output, f'{name}_{rank}_{index}.png'))
        nl[f'{rank}_{index}'] = tracks[k]['nl']
    json.dump(nl, open(os.path.join(output, f'{name}_nl.json'), 'w'))



In [28]:
results = json.load(open(json_file))
print(MRR(results))
print(recall(results, 5))
print(recall(results, 10))
worst5 = worst(results, 5)
tracks = json.load(open(json_tracks))

0.22137726494609586
0.32367149758454106
0.48792270531400966


In [21]:
save_analysis(worst5, tracks, 'worst')

In [16]:
best10 = best(results, 10)

In [17]:
save_analysis(best10, tracks, 'best')